<a href="https://colab.research.google.com/github/emekaphilian/AI-ML-Projects/blob/main/Amazon_Electronics_Recommendation_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing a library that is not in Colaboratory

To import a library that's not in Colaboratory by default, you can use `!pip install` or `!apt-get install`.

In [4]:
!pip install streamlit
!pip install pyngrok
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505219 sha256=c63ba0aa8f17a2d47fe635690682daa69b777d06eb0a929b85cd2f328b96457f
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import pandas as pd

# Check if the file exists after mounting Google Drive
dataset_path = "/content/drive/MyDrive/Projects/Amazon_Reviews/cleaned_amazon_reviews.csv"

# If path is correct, load the dataset
df = pd.read_csv(dataset_path)

# Display the first few rows of the dataset to confirm it's loaded
df.head()


,user_id,item_id,rating
0,0,0,5
1,1,0,1
2,2,0,3
3,3,0,2
4,4,0,1


In [10]:
from pathlib import Path
import pandas as pd # Import the pandas library

# Define the full content of the Colab notebook as a Jupyter Notebook (JSON structure)
notebook_content = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# 🎯 Amazon Electronics Recommender System (SVD + Streamlit in Colab)\n",
                "\n",
                "This notebook builds a collaborative filtering recommender system using **SVD** on the Amazon Electronics Review dataset and deploys it as a **Streamlit app in Colab using Pyngrok**."
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# ✅ Install required libraries\n",
                "!pip install streamlit pyngrok scikit-surprise"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# 📁 Load and preview the dataset\n",
                "import pandas as pd\n",
                "\n",
                "# Replace with your actual path or mount from Google Drive\n",
                "df = pd.read_csv(\"cleaned_amazon_reviews.csv\")\n",
                "df = df[['reviewerID', 'asin', 'overall', 'title']]\n",
                "df.head()"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# 🤖 Train SVD model (Collaborative Filtering)\n",
                "from surprise import SVD, Dataset, Reader\n",
                "\n",
                "reader = Reader(rating_scale=(1, 5))\n",
                "data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)\n",
                "trainset = data.build_full_trainset()\n",
                "\n",
                "svd = SVD()\n",
                "svd.fit(trainset)"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# 💾 Save the trained model\n",
                "import pickle\n",
                "with open(\"svd_model.pkl\", \"wb\") as f:\n",
                "    pickle.dump(svd, f)"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# 🌐 Write the Streamlit app code\n",
                "%%writefile app.py\n",
                "import streamlit as st\n",
                "import pandas as pd\n",
                "from surprise import SVD\n",
                "import pickle\n",
                "\n",
                "# Load data and model\n",
                "df = pd.read_csv(\"cleaned_amazon_reviews.csv\")\n",
                "model = pickle.load(open(\"svd_model.pkl\", \"rb\"))\n",
                "\n",
                "def get_recommendations(user_id):\n",
                "    all_item_ids = df['asin'].unique()\n",
                "    rated_items = df[df['reviewerID'] == user_id]['asin'].tolist()\n",
                "    items_to_predict = [item for item in all_item_ids if item not in rated_items]\n",
                "\n",
                "    predictions = []\n",
                "    for item in items_to_predict:\n",
                "        pred = model.predict(user_id, item)\n",
                "        title = df[df['asin'] == item]['title'].values[0] if not df[df['asin'] == item]['title'].empty else \"Unknown\"\n",
                "        predictions.append((title, pred.est))\n",
                "\n",
                "    predictions.sort(key=lambda x: x[1], reverse=True)\n",
                "    return predictions[:5]\n",
                "\n",
                "st.title(\"📦 Amazon Electronics Recommender\")\n",
                "\n",
                "user_id = st.text_input(\"Enter your Amazon user ID:\")\n",
                "if user_id:\n",
                "    st.subheader(\"Top Product Recommendations:\")\n",
                "    recommendations = get_recommendations(user_id)\n",
                "    for title, rating in recommendations:\n",
                "        st.write(f\"**{title}** — Predicted Rating: {rating:.2f}\")"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# 🚀 Launch the app with pyngrok\n",
                "from pyngrok import ngrok\n",
                "\n",
                "# Start Streamlit in the background\n",
                "!streamlit run app.py &\n",
                "\n",
                "# Expose the app with ngrok\n",
                "public_url = ngrok.connect(8501)\n",
                "print(\"Access your app at:\", public_url)"
            ]
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "name": "python"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 0
}


# Save as a downloadable notebook
cleaned_dataset_path = "/content/drive/MyDrive/Projects/Amazon_Reviews/cleaned_amazon_reviews.csv"
df_relevant = pd.read_csv(cleaned_dataset_path)

notebook_path = "/content/drive/MyDrive/Projects/Amazon_Reviews/Amazon_Electronics_Recommendation_Colab_Notebook.ipynb"
with open(notebook_path, "w") as f:
    import json
    json.dump(notebook_content, f)

# Output the notebook path
notebook_path # Changed from 'n' to 'notebook_path' to display the path

'/content/drive/MyDrive/Projects/Amazon_Reviews/Amazon_Electronics_Recommendation_Colab_Notebook.ipynb'